## Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
## Imports
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix,cohen_kappa_score
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np

import ipywidgets as widgets
from IPython.display import display

# 1. Without generalization

## 1.1 Import data

In [ ]:
Area = input("Area: ")
Year = input("Year: ")

Area: Pune
Year: 2024


In [ ]:
Ablation = input("Ablation(0 or 1): ")
feats = ''

Ablation(0 or 1): 0


In [ ]:
if Ablation == '0':
  df_train = pd.read_csv(f'/content/drive/MyDrive/DL - data/processed_data/training_processed_{Area}_{Year}.csv',index_col=0)
  df_test = pd.read_csv(f'/content/drive/MyDrive/DL - data/processed_data/testing_processed_{Area}_{Year}.csv',index_col=0)
else:
  feats = input("Features(s,si,st): ")
  df_train = pd.read_csv(f'/content/drive/MyDrive/DL - data/Ablation/training_{feats}_{Area}_{Year}.csv',index_col=0)
  df_test = pd.read_csv(f'/content/drive/MyDrive/DL - data/Ablation/testing_{feats}_{Area}_{Year}.csv',index_col=0)

In [ ]:
X_train = df_train.drop('Class', axis=1)
y_train = df_train['Class']
X_test = df_test.drop('Class', axis=1)
y_test = df_test['Class']

In [ ]:
df_train.head()

,Class,DVI,NDVI,RVI,SR_B2_contrast,SR_B2_diss,SR_B2_mean,SR_B2_var,SR_B3_contrast,SR_B3_diss,SR_B3_mean,SR_B3_var,SR_B4_contrast,SR_B4_diss,SR_B4_mean,SR_B4_var,SR_B5_contrast,SR_B5_diss,SR_B5_mean,SR_B5_var
0,0,0.624969,0.209230,0.653945,-0.189952,0.123622,-0.390689,-0.076377,-0.204564,0.101440,-0.193281,-0.109263,-0.044831,0.416091,-0.25054,0.146359,0.804268,1.278178,-0.206712,0.084637
1,0,0.624969,0.209230,0.653945,-0.198997,0.071651,-0.390689,-0.079524,-0.209337,0.062495,-0.193281,-0.108321,-0.058128,0.361416,-0.25054,0.133357,0.782581,1.169795,-0.206712,0.169166
2,0,0.010263,0.165852,0.715483,-0.193980,0.109360,-0.390689,-0.087459,-0.201927,0.114060,-0.193281,-0.120404,-0.048257,0.419294,-0.25054,0.097978,0.995130,1.365626,-0.206712,0.174543
3,0,0.007938,0.163758,0.718570,-0.190967,0.122529,-0.390689,-0.095699,-0.207726,0.087922,-0.193281,-0.122131,-0.045701,0.411226,-0.25054,0.132556,0.826823,1.276319,-0.206712,0.083055
4,0,0.007938,0.163758,0.718570,-0.199156,0.072745,-0.390689,-0.089177,-0.213734,0.044927,-0.193281,-0.123386,-0.061778,0.350079,-0.25054,0.118774,0.802470,1.156747,-0.206712,0.127379


## 1.2 Creating models

### 1.2.1. Random forest

In [ ]:
param_grid = {
    'n_estimators': [50, 100, 150, 200]
}

rf_model = RandomForestClassifier()
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, scoring='accuracy',n_jobs=-1)
grid_search.fit(X_train, y_train)

print("Best parameters found: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

rf_model = grid_search.best_estimator_

Best parameters found:  {'n_estimators': 200}
Best score:  0.9657428029234747


In [ ]:
# rf_model = RandomForestClassifier(n_estimators=150, random_state=42)

In [ ]:
rf_model.fit(X_train,y_train)

RandomForestClassifier(n_estimators=150, random_state=42)

### 1.2.2. Support Vector Machines

In [ ]:
# Define the parameter grid for SVC
param_grid_svc = {
    'C': [0.1, 1, 10, 100],  # Regularization parameter
    'kernel': ['linear', 'rbf', 'poly'],  # Kernel type
    'gamma': ['scale', 'auto', 0.1, 1],  # Kernel coefficient
}

# Create an SVC model
svc_model = SVC()

# Create a GridSearchCV object
grid_search_svc = GridSearchCV(estimator=svc_model, param_grid=param_grid_svc, cv=5, scoring='accuracy',n_jobs=-1)

# Fit the model to the training data
grid_search_svc.fit(X_train, y_train)

# Print the best parameters and score
print("Best parameters for SVC found: ", grid_search_svc.best_params_)
print("Best score for SVC: ", grid_search_svc.best_score_)

# Get the best estimator
svm = grid_search_svc.best_estimator_

Best parameters for SVC found:  {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}
Best score for SVC:  0.9765367088648806


In [ ]:
# svm = SVC(C=1, gamma=0.1, kernel='poly')

In [ ]:
svm.fit(X_train,y_train)

SVC(C=1, gamma=0.1, kernel='poly')

## 1.2.3 Naive Bayes

In [ ]:
nb = GaussianNB()

In [ ]:
nb.fit(X_train, y_train)

GaussianNB()

## 1.2.4 Adaboost

In [ ]:
# Define the parameter grid for AdaBoost
param_grid_ada = {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.01, 0.1, 1.0],
    'algorithm': ['SAMME']
}

# Create an AdaBoost model
ada_model = AdaBoostClassifier()

# Create a GridSearchCV object
grid_search_ada = GridSearchCV(estimator=ada_model, param_grid=param_grid_ada, cv=5, scoring='accuracy',n_jobs=-1,verbose=1)

# Fit the model to the training data
grid_search_ada.fit(X_train, y_train)

# Print the best parameters and score
print("Best parameters for AdaBoost found: ", grid_search_ada.best_params_)
print("Best score for AdaBoost: ", grid_search_ada.best_score_)

# Get the best estimator
ada = grid_search_ada.best_estimator_

Fitting 5 folds for each of 12 candidates, totalling 60 fits


/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The parameter 'algorithm' is deprecated in 1.6 and has no effect. It will be removed in version 1.8.
  warnings.warn(


Best parameters for AdaBoost found:  {'algorithm': 'SAMME', 'learning_rate': 0.1, 'n_estimators': 200}
Best score for AdaBoost:  0.9472807428578639


In [ ]:
# ada = AdaBoostClassifier(n_estimators= 200,learning_rate= 0.1,algorithm = 'SAMME')

In [ ]:
ada.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The parameter 'algorithm' is deprecated in 1.6 and has no effect. It will be removed in version 1.8.
  warnings.warn(


AdaBoostClassifier(algorithm='SAMME', learning_rate=0.1, n_estimators=200)

# 1.2.5 KNN

In [ ]:
# Define the parameter grid for KNN
param_grid_neigh = {
    'n_neighbors': [20,25,30,35,40,50,55,60,70,80],
    'weights': ["uniform", "distance"],
    'algorithm': ['auto']
}

# Creating a classifier
neigh = KNeighborsClassifier()

# Creating a GridCV object
grid_search_knn = GridSearchCV(estimator=neigh, param_grid=param_grid_neigh, cv=5, scoring='accuracy',n_jobs=-1,verbose=True)

# Fitting the model to the training data
grid_search_knn.fit(X_train, y_train)

# Print the best parameters and score
print("Best parameters for KNN found: ", grid_search_knn.best_params_)
print("Best score for KNN: ", grid_search_knn.best_score_)

# Get the best estimator
ada = grid_search_knn.best_estimator_

In [ ]:
# neigh = KNeighborsClassifier(n_neighbors=70)
# neigh.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=70)

## 1.3 Metrics

In [ ]:
def calculate_per_class_accuracy(y_true, y_pred):
  """
  Calculates user and producer accuracy for each class.

  Args:
    y_true: True labels.
    y_pred: Predicted labels.

  Returns:
    A dictionary containing user and producer accuracies for each class.
  """
  cm = confusion_matrix(y_true, y_pred)
  print(cm)
  class_names = np.unique(y_true)
  results = {}
  for i, class_name in enumerate(class_names):
      tp = cm[i, i]
      fp = np.sum(cm[:, i]) - tp
      fn = np.sum(cm[i, :]) - tp
      tn = np.sum(cm) - tp - fp - fn

      producer_accuracy = tp / (tp + fn) if (tp + fn) > 0 else 0
      user_accuracy = tp / (tp + fp) if (tp + fp) > 0 else 0
      results[class_name] = {
          'producer_accuracy': producer_accuracy,
          'user_accuracy': user_accuracy
      }
  return results,cm

In [ ]:
y_pred_rf_train = rf_model.predict(X_train)
y_pred_svm_train = svm.predict(X_train)
y_pred_nb_train = nb.predict(X_train)
y_pred_ada_train = ada.predict(X_train)
y_pred_knn_train = neigh.predict(X_train)

In [ ]:
y_pred_rf_test = rf_model.predict(X_test)
y_pred_svm_test = svm.predict(X_test)
y_pred_nb_test = nb.predict(X_test)
y_pred_ada_test = ada.predict(X_test)
y_pred_knn_test = neigh.predict(X_test)

In [ ]:
# Create dataframes to store metrics
metrics_df_train = pd.DataFrame(columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1-Score','Class_accuracies','Confusion_matrix'])
metrics_df_test = pd.DataFrame(columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1-Score','Class_accuracies','Confusion_matrix'])

In [ ]:
def calculate_metrics(y_true, y_pred, model_name):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    class_accuracies,cm = calculate_per_class_accuracy(y_true, y_pred)
    return pd.DataFrame({'Model': [model_name], 'Accuracy': [accuracy], 'Precision': [precision], 'Recall': [recall], 'F1-Score': [f1],'Class_accuracies':[class_accuracies],'Confusion_matrix':[cm]})

In [ ]:
metrics_df_train = pd.concat([metrics_df_train, calculate_metrics(y_train, y_pred_rf_train, 'Random Forest')], ignore_index=True)
metrics_df_train = pd.concat([metrics_df_train, calculate_metrics(y_train, y_pred_svm_train, 'SVM')], ignore_index=True)
metrics_df_train = pd.concat([metrics_df_train, calculate_metrics(y_train, y_pred_nb_train, 'Naive Bayes')], ignore_index=True)
metrics_df_train = pd.concat([metrics_df_train, calculate_metrics(y_train, y_pred_ada_train, 'AdaBoost')], ignore_index=True)
metrics_df_train = pd.concat([metrics_df_train, calculate_metrics(y_train, y_pred_knn_train, 'KNN')], ignore_index=True)

[[30615     0     0     0]
 [    0 16071     0     0]
 [    0     0  6672     0]
 [    0     0     0  5188]]
[[30483    89    38     5]
 [   78 15901     0    92]
 [  138    67  6419    48]
 [  109   606     1  4472]]
[[28475  1148   715   277]
 [   40 10842     0  5189]
 [   85     0  6359   228]
 [   53   286    95  4754]]
[[29676   447   416    76]
 [  139 15452     3   477]
 [  146     6  6357   163]
 [   33  1344     4  3807]]
[[30398   127    47    43]
 [   72 15880     2   117]
 [  148    59  6388    77]
 [   57   427     5  4699]]


<ipython-input-176-02da4a327711>:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metrics_df_train = pd.concat([metrics_df_train, calculate_metrics(y_train, y_pred_rf_train, 'Random Forest')], ignore_index=True)


In [ ]:
metrics_df_test = pd.concat([metrics_df_test, calculate_metrics(y_test, y_pred_rf_test, 'Random Forest')], ignore_index=True)
metrics_df_test = pd.concat([metrics_df_test, calculate_metrics(y_test, y_pred_svm_test, 'SVM')], ignore_index=True)
metrics_df_test = pd.concat([metrics_df_test, calculate_metrics(y_test, y_pred_nb_test, 'Naive Bayes')], ignore_index=True)
metrics_df_test = pd.concat([metrics_df_test, calculate_metrics(y_test, y_pred_ada_test, 'AdaBoost')], ignore_index=True)
metrics_df_test = pd.concat([metrics_df_test, calculate_metrics(y_test, y_pred_knn_test, 'KNN')], ignore_index=True)

[[7766  101   12   13]
 [  11 3250    2   84]
 [ 194    1  791   12]
 [   3   49    0 1222]]
[[7744  140    5    3]
 [   0 3247    6   94]
 [ 267    5  724    2]
 [   5  120    1 1148]]
[[7328  291   58  215]
 [   0 2609    0  738]
 [ 195    0  785   18]
 [   4   39   42 1189]]
[[7452  266   48  126]
 [   7 3135    2  203]
 [ 194    0  712   92]
 [   3  277    3  991]]
[[7649  144   42   57]
 [   0 3235    3  109]
 [ 260    6  731    1]
 [   2   84    7 1181]]


<ipython-input-177-081b093f658a>:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metrics_df_test = pd.concat([metrics_df_test, calculate_metrics(y_test, y_pred_rf_test, 'Random Forest')], ignore_index=True)


## 1.4 Save the metrics

In [ ]:
feats

''

In [ ]:
# Save metrics to a CSV file
if feats == 's':
  feats = 'Spectral_features'
elif feats == 'st':
  feats = 'Spectral+Textures'
elif feats == 'si':
  feats = 'Spectral+Indices'
else:
  feats = 'Non-Ablated_Data'

print(f'/content/drive/MyDrive/Results/ML/Non-generalizability/{feats}/{Area}_{Year}_train.csv')

/content/drive/MyDrive/Results/ML/Non-generalizability/Non-Ablated_Data/Chicago_2024_train.csv


# 2. Generalizability study

## 2.1 Importing data

In [ ]:
training_area = input("Training area: ")
training_year = input("Training year: ")
testing_area = input("Testing area: ")
testing_year = input("Testing year:")

In [ ]:
Ablation = input("Ablation(0 or 1): ")
feats = ''

In [ ]:
if Ablation == '0':
  df_train = pd.read_csv(f'/content/drive/MyDrive/DL - data/processed_data/training_processed_{training_area}_{training_year}.csv',index_col=0)
  df_cv = pd.read_csv(f'/content/drive/MyDrive/DL - data/processed_data/testing_processed_{training_area}_{training_year}.csv',index_col=0)
  df_test = pd.read_csv(f'/content/drive/MyDrive/DL - data/processed_data/processed_gz_{testing_area}_{testing_year}.csv',index_col=0)
else:
  feats = input("Features(s,si,st): ")
  df_train = pd.read_csv(f'/content/drive/MyDrive/DL - data/Ablation/training_{feats}_{training_area}_{training_year}.csv',index_col=0)
  df_cv = pd.read_csv(f'/content/drive/MyDrive/DL - data/Ablation/testing_{feats}_{training_area}_{training_year}.csv',index_col=0)
  df_test = pd.read_csv(f'/content/drive/MyDrive/DL - data/Ablation/gz_{feats}_{testing_area}_{testing_year}.csv',index_col=0)

In [ ]:
X_train = df_train.drop('Class', axis=1)
y_train = df_train['Class']
X_cv  = df_cv.drop('Class', axis=1)
y_cv = df_cv['Class']
X_test = df_test.drop('Class', axis=1)
y_test = df_test['Class']

## 2.2 Creating and training models

### 2.2.1 Random forests

In [ ]:
# param_grid = {
#     'n_estimators': [50, 100, 150, 200]
# }

# rf_model = RandomForestClassifier()
# grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, scoring='accuracy')
# grid_search.fit(X_train, y_train)

# print("Best parameters found: ", grid_search.best_params_)
# print("Best score: ", grid_search.best_score_)

# rf_model = grid_search.best_estimator_

In [ ]:
rf_model = RandomForestClassifier(n_estimators=200, random_state=42)

In [ ]:
rf_model.fit(X_train,y_train)

### 2.2.2 Support vector machines

In [ ]:
# # Define the parameter grid for SVC
# param_grid_svc = {
#     'C': [0.1, 1, 10, 100],  # Regularization parameter
#     'kernel': ['linear', 'rbf', 'poly'],  # Kernel type
#     'gamma': ['scale', 'auto', 0.1, 1],  # Kernel coefficient
# }

# # Create an SVC model
# svc_model = SVC()

# # Create a GridSearchCV object
# grid_search_svc = GridSearchCV(estimator=svc_model, param_grid=param_grid_svc, cv=5, scoring='accuracy')

# # Fit the model to the training data
# grid_search_svc.fit(X_train, y_train)

# # Print the best parameters and score
# print("Best parameters for SVC found: ", grid_search_svc.best_params_)
# print("Best score for SVC: ", grid_search_svc.best_score_)

# # Get the best estimator
# svm = grid_search_svc.best_estimator_

In [ ]:
svm = SVC(C=0.1, gamma='scale', kernel='linear')

In [ ]:
svm.fit(X_train,y_train)

In [ ]:
nb = GaussianNB()

In [ ]:
nb.fit(X_train, y_train)

In [ ]:
# # Define the parameter grid for AdaBoost
# param_grid_ada = {
#     'n_estimators': [50, 100, 150],
#     'learning_rate': [0.01, 0.1, 1.0],
#     'algorithm': ['SAMME', 'SAMME.R']
# }

# # Create an AdaBoost model
# ada_model = AdaBoostClassifier()

# # Create a GridSearchCV object
# grid_search_ada = GridSearchCV(estimator=ada_model, param_grid=param_grid_ada, cv=5, scoring='accuracy')

# # Fit the model to the training data
# grid_search_ada.fit(X_train, y_train)

# # Print the best parameters and score
# print("Best parameters for AdaBoost found: ", grid_search_ada.best_params_)
# print("Best score for AdaBoost: ", grid_search_ada.best_score_)

# # Get the best estimator
# ada = grid_search_ada.best_estimator_

In [ ]:
ada = AdaBoostClassifier(n_estimators= 150,learning_rate= 0.1,algorithm = 'SAMME')

In [ ]:
ada.fit(X_train, y_train)

In [ ]:
def calculate_per_class_accuracy(y_true, y_pred):
  """
  Calculates user and producer accuracy for each class.

  Args:
    y_true: True labels.
    y_pred: Predicted labels.

  Returns:
    A dictionary containing user and producer accuracies for each class.
  """
  cm = confusion_matrix(y_true, y_pred)
  print(cm)
  class_names = np.unique(y_true)
  results = {}
  for i, class_name in enumerate(class_names):
      tp = cm[i, i]
      fp = np.sum(cm[:, i]) - tp
      fn = np.sum(cm[i, :]) - tp
      tn = np.sum(cm) - tp - fp - fn

      producer_accuracy = tp / (tp + fn) if (tp + fn) > 0 else 0
      user_accuracy = tp / (tp + fp) if (tp + fp) > 0 else 0
      results[class_name] = {
          'producer_accuracy': producer_accuracy,
          'user_accuracy': user_accuracy
      }
  return results,cm

In [ ]:
# Predictions on train data
y_pred_rf_train = rf_model.predict(X_train)
y_pred_svm_train = svm.predict(X_train)
y_pred_nb_train = nb.predict(X_train)
y_pred_ada_train = ada.predict(X_train)

In [ ]:
# Predictions on cv data
y_pred_rf_cv = rf_model.predict(X_cv)
y_pred_svm_cv = svm.predict(X_cv)
y_pred_nb_cv = nb.predict(X_cv)
y_pred_ada_cv = ada.predict(X_cv)

In [ ]:
# Predictions on test data
y_pred_rf_test = rf_model.predict(X_test)
y_pred_svm_test = svm.predict(X_test)
y_pred_nb_test = nb.predict(X_test)
y_pred_ada_test = ada.predict(X_test)

In [ ]:
# Create dataframes to store metrics
metrics_df_train = pd.DataFrame(columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1-Score','Class_accuracies','Confusion_matrix'])
metrics_df_cv = pd.DataFrame(columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1-Score','Class_accuracies','Confusion_matrix'])
metrics_df_test = pd.DataFrame(columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1-Score','Class_accuracies','Confusion_matrix'])

In [ ]:
def calculate_metrics(y_true, y_pred, model_name):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    class_accuracies,cm = calculate_per_class_accuracy(y_true, y_pred)
    return pd.DataFrame({'Model': [model_name], 'Accuracy': [accuracy], 'Precision': [precision], 'Recall': [recall], 'F1-Score': [f1],'Class_accuracies':[class_accuracies],'Confusion_matrix':[cm]})



In [ ]:
metrics_df_train = pd.concat([metrics_df_train, calculate_metrics(y_train, y_pred_rf_train, 'Random Forest')], ignore_index=True)
metrics_df_train = pd.concat([metrics_df_train, calculate_metrics(y_train, y_pred_svm_train, 'SVM')], ignore_index=True)
metrics_df_train = pd.concat([metrics_df_train, calculate_metrics(y_train, y_pred_nb_train, 'Naive Bayes')], ignore_index=True)
metrics_df_train = pd.concat([metrics_df_train, calculate_metrics(y_train, y_pred_ada_train, 'AdaBoost')], ignore_index=True)

In [ ]:
metrics_df_cv = pd.concat([metrics_df_cv, calculate_metrics(y_cv, y_pred_rf_cv, 'Random Forest')], ignore_index=True)
metrics_df_cv = pd.concat([metrics_df_cv, calculate_metrics(y_cv, y_pred_svm_cv, 'SVM')], ignore_index=True)
metrics_df_cv = pd.concat([metrics_df_cv, calculate_metrics(y_cv, y_pred_nb_cv, 'Naive Bayes')], ignore_index=True)
metrics_df_cv = pd.concat([metrics_df_cv, calculate_metrics(y_cv, y_pred_ada_cv, 'AdaBoost')], ignore_index=True)

In [ ]:
metrics_df_test = pd.concat([metrics_df_test, calculate_metrics(y_test, y_pred_rf_test, 'Random Forest')], ignore_index=True)
metrics_df_test = pd.concat([metrics_df_test, calculate_metrics(y_test, y_pred_svm_test, 'SVM')], ignore_index=True)
metrics_df_test = pd.concat([metrics_df_test, calculate_metrics(y_test, y_pred_nb_test, 'Naive Bayes')], ignore_index=True)
metrics_df_test = pd.concat([metrics_df_test, calculate_metrics(y_test, y_pred_ada_test, 'AdaBoost')], ignore_index=True)

In [ ]:
# Save metrics to a CSV file
if feats == 's':
  feats = 'Spectral_features'
elif feats == 'st':
  feats = 'Spectral+Textures'
elif feats == 'si':
  feats = 'Spectral+Indices'
else:
  feats = 'Non-Ablated_Data'

metrics_df_train.to_csv(f'/content/drive/MyDrive/Results/ML/Generalizability/{feats}/gz_{training_area}_{training_year}_{testing_area}_{testing_year}_train.csv', index=False)
metrics_df_cv.to_csv(f'/content/drive/MyDrive/Results/ML/Generalizability/{feats}/gz_{training_area}_{training_year}_{testing_area}_{testing_year}_cv.csv', index=False)
metrics_df_test.to_csv(f'/content/drive/MyDrive/Results/ML/Generalizability/{feats}/gz_{training_area}_{training_year}_{testing_area}_{testing_year}_test.csv', index=False)